In [4]:

import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import tensorflow.keras as keras

In [5]:
x = pd.read_csv("Cary_Weather_Data.csv")
x['DATE'] = pd.to_datetime(x[['YEAR', 'MONTH', 'DAY']])
x = x.drop(columns=['YEAR','MONTH','DAY'])
#x = x.drop(columns=['Humidity','Precipitation','Wind Speed'])
x = x[['DATE', 'Temperature', 'Humidity', 'Precipitation', 'Wind Speed']]
x.head()
x.set_index('DATE', inplace=True)
x.head()


,Temperature,Humidity,Precipitation,Wind Speed
DATE,,,,
1981-07-01,15.12,10.68,6.65,3.59
1981-07-02,7.53,13.12,10.68,1.64
1981-07-03,7.32,13.79,37.42,2.43
1981-07-04,10.43,14.40,8.41,2.29
1981-07-05,13.13,15.20,4.13,1.58


In [6]:
scaler = StandardScaler()
x_scale = scaler.fit_transform(x)
x_scale = pd.DataFrame(x_scale, columns=x.columns, index=x.index)
x_scale.head()

,Temperature,Humidity,Precipitation,Wind Speed
DATE,,,,
1981-07-01,0.926990,0.385333,0.478536,1.918945
1981-07-02,-1.305674,0.925994,1.044274,-0.352127
1981-07-03,-1.367448,1.074455,4.798080,0.567948
1981-07-04,-0.452614,1.209620,0.725608,0.404897
1981-07-05,0.341614,1.386887,0.124774,-0.422006


In [7]:
def create_dataset(X, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(X.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [8]:
time_steps = 1

X, y = create_dataset(x_scale, time_steps)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123, shuffle=False)
X_train.shape
X_val.shape

(3106, 1, 4)

In [10]:
model = tf.keras.models.Sequential([
    # LSTM Layer 1 with return_sequences=True
    keras.layers.LSTM(256, input_shape=(time_steps, 4), return_sequences=True, name='lstm_layer_1'),
    keras.layers.Dropout(0.2, name='dropout_layer_1'),
    
    # LSTM Layer 2 with return_sequences=True
    keras.layers.LSTM(128, return_sequences=True, name='lstm_layer_2'),
    keras.layers.Dropout(0.2, name='dropout_layer_2'),
    
    # Bidirectional LSTM Layer with return_sequences=True
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True), name='bidirectional_lstm_layer'),
    keras.layers.Dropout(0.2, name='dropout_layer_3'),
    
    # Attention Layer
    #keras.layers.Attention(name='attention_layer'),
    
    # Global Max Pooling 1D Layer
    keras.layers.GlobalMaxPooling1D(name='global_max_pooling_layer'),
    
    # Dense Layer 1
    keras.layers.Dense(128, activation='relu', name='dense_layer_1'),
    keras.layers.Dropout(0.3, name='dropout_layer_4'),
    
    # Dense Layer 2
    keras.layers.Dense(64, activation='relu', name='dense_layer_2'),
    
    # Output Layer with Linear Activation
    keras.layers.Dense(1, activation='linear', name='output_layer')
])



cp = tf.keras.callbacks.ModelCheckpoint('model', save_best_only=True)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_layer_1 (LSTM)         (None, 1, 256)            267264    
                                                                 
 dropout_layer_1 (Dropout)   (None, 1, 256)            0         
                                                                 
 lstm_layer_2 (LSTM)         (None, 1, 128)            197120    
                                                                 
 dropout_layer_2 (Dropout)   (None, 1, 128)            0         
                                                                 
 bidirectional_lstm_layer (  (None, 1, 128)            98816     
 Bidirectional)                                                  
                                                                 
 dropout_layer_3 (Dropout)   (None, 1, 128)            0         
                                                      

In [13]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# Compile the model with Mean Squared Error (MSE) loss and Mean Absolute Error (MAE) metric
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])


In [14]:
cp = tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True)
history = model.fit(X_train, y_train, epochs=10, callbacks=cp)

Epoch 1/10
389/389 [==============================] - 6s 10ms/step - loss: 0.9390 - mae: 0.7223
Epoch 2/10
389/389 [==============================] - 3s 8ms/step - loss: 0.9384 - mae: 0.7243
Epoch 3/10
389/389 [==============================] - 3s 9ms/step - loss: 0.9386 - mae: 0.7241
Epoch 4/10
389/389 [==============================] - 4s 9ms/step - loss: 0.9384 - mae: 0.7246
Epoch 5/10
104/389 [=======>......................] - ETA: 2s - loss: 0.9435 - mae: 0.7232

KeyboardInterrupt: 

In [ ]:
matplotlib.interactive(True)
%matplotlib qt
plt.plot(history.history['loss'], label='Training Loss')
#plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
prediction = model.predict(X_val)
prediction_reshape = prediction[:, 0]
x_val_reshz = X_val[:, 0, 0]

98/98 [==============================] - 0s 2ms/step


In [ ]:
time_val = x.index[-len(X_val):]

In [ ]:
plt.plot(x_scale['Temperature'], color='green')
plt.plot(time_val, x_val_reshz, label='Actual', color='blue')
plt.plot(time_val, prediction_reshape, label='Prediction', color='orange')
plt.show()